In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as T
import torchvision.datasets

In [21]:
batch_size = 10 #256
data_mean = 0.128
data_std = 0.305

In [3]:
# amplitude for the noise augmentation
augm_sigma = 0.08
data_dir = 'D:/_0Luciano/_0PHD/datasets' #'mnist_data'

In [4]:
def unnormalize(x):
    '''go from normaized data x back to the original range'''
    return x * data_std + data_mean

In [5]:
transf =  T.Compose([T.ToTensor(), T.Normalize(data_mean, data_std, inplace=False)  ])

train_data = torchvision.datasets.MNIST(data_dir, train=True, download=True, transform=transf)
test_data  = torchvision.datasets.MNIST(data_dir, train=False, download=True, transform=transf)

d:\simbr-env\Miniconda\envs\salt\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# Sample a fixed batch of 1024 validation examples
val_x, val_l = zip(*list(train_data[i] for i in range(1024)))
val_x = torch.stack(val_x, 0).cuda()
val_l = torch.LongTensor(val_l).cuda()

In [7]:
def noise(x):
    return x + augm_sigma * torch.randn_like(x)

In [22]:
# Exclude the validation batch from the training data
train_data.data = train_data.data[1024:]
train_data.targets = train_data.targets[1024:]
# Add the noise-augmentation to the (non-validation) training data:
train_data.transform = T.Compose([train_data.transform, T.Lambda(noise) ])                            #lambda x: x + augm_sigma * torch.randn_like(x)])

train_loader  = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
test_loader   = DataLoader(test_data,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)

In [25]:
train_data.data.shape

torch.Size([56928, 28, 28])

In [9]:
from time import time
from tqdm import tqdm
import torch
import torch.nn
import torch.optim
import numpy as np


In [10]:

import model


In [ ]:
import data


In [23]:
it=iter(train_loader)
real_batch = next(iter(it))


RuntimeError: DataLoader worker (pid(s) 10540, 16236, 11784, 17908) exited unexpectedly

In [14]:
cinn = model.MNIST_cINN(5e-4)
cinn.cuda()
scheduler = torch.optim.lr_scheduler.MultiStepLR(cinn.optimizer, milestones=[20, 40], gamma=0.1)

N_epochs = 60
t_start = time()
nll_mean = []

In [17]:
print('Epoch\tBatch/Total \tTime \tNLL train\tNLL val\tLR')
for epoch in range(N_epochs):
    for i, (x, l) in enumerate(train_loader): #data.train_loader):
        x, l = x.cuda(), l.cuda()
        z, log_j = cinn(x, l)

        nll = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total
        nll.backward()
        torch.nn.utils.clip_grad_norm_(cinn.trainable_parameters, 10.)
        nll_mean.append(nll.item())
        cinn.optimizer.step()
        cinn.optimizer.zero_grad()

        if not i % 50:
            with torch.no_grad():
                z, log_j = cinn(val_x, val_l) #cinn(data.val_x, data.val_l)
                nll_val = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total

            print('%.3i \t%.5i/%.5i \t%.2f \t%.6f\t%.6f\t%.2e' % (epoch,
                                                            i, len(data.train_loader),
                                                            (time() - t_start)/60.,
                                                            np.mean(nll_mean),
                                                            nll_val.item(),
                                                            cinn.optimizer.param_groups[0]['lr'],
                                                            ), flush=True)
            nll_mean = []
    scheduler.step()

Epoch	Batch/Total 	Time 	NLL train	NLL val	LR


RuntimeError: DataLoader worker (pid(s) 28324, 17372, 2288, 20632) exited unexpectedly

In [ ]:
torch.save(cinn.state_dict(), 'output/mnist_cinn.pt')